In [1]:
import pandas as pd
from datetime import datetime, timedelta

import zipfile
from shapely.geometry import Point

import pyproj
import math

from collections import defaultdict

In [2]:
df = pd.read_excel('Report_JR14_Vehicle History Report - data.xlsx',skiprows=10)

In [3]:
'''
    Create Trips from the GPS Data
    1. 
'''
# df.columns

'\n    Create Trips from the GPS Data\n    1. \n'

In [4]:
depot = [-27.426770, 153.150956]
proj_wgs84 = pyproj.Proj(proj='latlong', datum='WGS84')

ts = '2023-01-02T12:03:03.323'
st_ts = '2023-01-01T00:00:00.000'
def convert_to_gtfs_time(ts,st_ts):
    ts_dt = datetime.strptime(ts, "%Y-%m-%dT%H:%M:%S.%f")
    start_trip_dt = pd.to_datetime(st_ts)
    
    ts_dt_obj = ts_dt.date()
    start_trip_dt_obj = start_trip_dt.date()

    if ts_dt_obj == start_trip_dt_obj:
        return ((ts_dt.hour * 60) + ts_dt.minute + ts_dt.second)
    elif (ts_dt - start_trip_dt).days > 0:
        return (((ts_dt - start_trip_dt).seconds) // 60 + ((ts_dt - start_trip_dt).days * 24 * 60 * 60) // 60)
    else:
        print("Error in Timestamp")
        return None

def check_buffer(point1,point2):
    # Convert Shapely Point objects to (longitude, latitude) tuples
    lon1, lat1 = point1.x, point1.y
    lon2, lat2 = point2.x, point2.y

    # Convert latitude and longitude from degrees to radians
    lat1_rad, lon1_rad = math.radians(lat1), math.radians(lon1)
    lat2_rad, lon2_rad = math.radians(lat2), math.radians(lon2)

    # Calculate the distance using pyproj's Geod class
    geod = pyproj.Geod(ellps='WGS84')
    azimuth1, azimuth2, distance = geod.inv(lon1_rad, lat1_rad, lon2_rad, lat2_rad)
    return distance

In [20]:
# session_starts = []

current_session_start = None
on_odometer = None
off_odometer = None

trip_ends = defaultdict(list)
calendar = defaultdict(list)
stops = defaultdict(list)
stop_times = defaultdict(list)
trips = defaultdict(list)
routes = defaultdict(list)
shapes = defaultdict(list)
feed_info = defaultdict(list)
agency = defaultdict(list)

blocks_for_input = {"blocks":[]}

df = df[df['Reason'].isin(['Arr. Way Point','Break End','Break Start','Dep. Way Point','Logoff','Logon','Status Report','Ignition Off','Ignition On','Odometer Offset'])]

agency["agency_id"].append("MGL")
agency["agency_name"].append("Microgrid Labs")
agency["agency_url"].append("https://www.microgridlabs.com")
agency["agency_timezone"].append("Asia/Kolkata")

calendar["start_date"].append(pd.to_datetime(df['GPS'].min()).strftime('%Y%m%d'))
calendar["end_date"].append(pd.to_datetime(df['GPS'].max()).strftime('%Y%m%d'))
calendar["monday"].append("1")
calendar["tuesday"].append("1")
calendar["wednesday"].append("1")
calendar["thursday"].append("1")
calendar["friday"].append("1")
calendar["saturday"].append("1")
calendar["sunday"].append("1")
calendar["service_id"].append("S1")

feed_info["feed_publisher_name"].append("Microgrid Labs")
feed_info["feed_publisher_url"].append("https://www.microgridlabs.com")
feed_info["feed_lang"].append("eng")
feed_info["feed_start_date"].append("20230101")
feed_info["feed_end_date"].append("20250101")
feed_info["feed_version"].append("1")
feed_info["feed_url"].append("https://www.microgridlabs.com")

trip_no = 1
block_id = 1
prev_start , prev_stop = None, None
for index, row in df.iterrows():
    if row['Reason'] == 'Ignition On':
        current_session_start = index
        on_odometer = row['Odometer']
    elif row['Reason'] == 'Ignition Off' and current_session_start is not None:
        off_odometer = row['Odometer']

        if on_odometer is not None and off_odometer is not None:
            # Set another logic for selecting trips since SaaS only takes trips < 1km in distance for REM
            diff = off_odometer - on_odometer
            if diff > 1:
                
                trips_df = df.loc[current_session_start:index].reset_index()

                # Set Trips file here
                trips["trip_id"].append(f"T_{trip_no}")
                trips["route_id"].append(f"R_{trip_no}")
                trips["service_id"].append("S1")

                trips["shape_id"].append(f"Sh_{trip_no}")
                trips["trip_short_name"].append(f"Short Name {trip_no}")
                trips["direction_id"].append("1")

                start_point  = Point(trips_df['Latitude'][0],trips_df['Longitude'][0])
                if trip_no > 1 :
                    start_point  = Point(trips_df['Latitude'][0],trips_df['Longitude'][0])

                # Set Block Logic here
                # Block logic can also be setup using df.[prev_start and prev_stop] and df.[current_session_start and index]
                if check_buffer(Point(depot[0],depot[1]),start_point) <= 10 and (prev_start is not None and prev_stop is not None) and (prev_start <= len(df)):
                    prev_t_start = df.iloc[prev_start]
                    prev_t_end = df.iloc[prev_stop]
                    if (prev_t_end['GPS'] - trips_df['GPS'][0]).seconds < 0:
                        print(prev_t_end['GPS'] - trips_df['GPS'][0])
                    else:
                        diff = prev_t_end['GPS'] - trips_df['GPS'][0]
                        print(f"Diff{str(diff)}")
                    block_id+=1

                trips["block_id"].append(f"B{block_id}")
                # Set Routes file here
                routes["agency_id"].append("MGL")
                routes["route_id"].append(f"R_{trip_no}")
                routes["route_short_name"].append(f"RSn {trip_no}")
                routes["route_long_name"].append(f"Long Name {trip_no}")
                routes["route_type"].append(3)

                prev_start = current_session_start
                prev_stop = index

                trip_start_day = trips_df['GPS'][0].strftime('%Y%m%d')

                for id,trips_row in trips_df.iterrows():    

                    stop_id = round(abs(float(trips_row['Latitude'])),2).__str__() + round(abs(float(trips_row['Longitude'])),2).__str__() + pd.to_datetime(trips_row['GPS']).strftime('%Y%m%d%H%M%S') + id.__str__()

                    # Set Stops File - DONE
                    if id in [0, len(trips_df) - 1]:
                        stops["stop_id"].append(f"{stop_id}")
                        stops["stop_name"].append(f"Stop {id}")
                        stops["stop_lat"].append(trips_row['Latitude'])
                        stops["stop_lon"].append(trips_row['Longitude'])

                        # Set Stop Times File - DONE
                        stop_times["stop_id"].append(f"{stop_id}")
                        stop_times["trip_id"].append(f"T_{trip_no}")

                        # Timepoint is required for start_time and end_time
                        stop_times["timepoint"].append("1")
                        if id == 0:
                            stop_times["stop_sequence"].append("1")
                        else:
                            stop_times["stop_sequence"].append("2")

                        stop_times["arrival_time"].append(f"{convert_to_gtfs_time(trips_row['GPS'].strftime('%Y-%m-%dT%H:%M:%S.%f'),trip_start_day)}")
                        stop_times["departure_time"].append(f"{convert_to_gtfs_time(trips_row['GPS'].strftime('%Y-%m-%dT%H:%M:%S.%f'),trip_start_day)}")

                    # Set Shapes File - DONE
                    shapes["shape_id"].append(f"Sh_{trip_no}")
                    shapes["shape_pt_lat"].append(trips_row['Latitude'])
                    shapes["shape_pt_lon"].append(trips_row['Longitude'])
                    shapes["shape_pt_sequence"].append(f"{int(id)+1}")

                # Proceed to next Trip ID
                trip_no += 1

        else:
            # Do something when the Ignition On or Off row does not have any odometer value
            print("Ignition On or Off do not have an Odometer Value")

Diff0 days 00:01:07
Diff-1 days +08:30:59
Diff0 days 00:06:46
Diff0 days 00:09:00
Diff-1 days +11:58:28
Diff0 days 00:16:05
Diff0 days 00:07:27
Diff0 days 00:15:12
Diff0 days 00:15:06
Diff0 days 00:14:00
Diff0 days 17:31:36
Diff0 days 00:11:04
Diff0 days 00:24:10
Diff0 days 00:29:50
Diff0 days 00:24:00
Diff-1 days +11:26:13
Diff0 days 00:54:58
Diff0 days 00:56:14
Diff0 days 01:16:04
Diff0 days 03:04:42
Diff0 days 12:19:07
Diff0 days 02:06:06
Diff0 days 01:44:56
Diff0 days 01:28:02
Diff0 days 00:29:05
Diff0 days 02:08:49
Diff0 days 00:40:40
Diff0 days 02:45:49
Diff0 days 12:20:09
Diff0 days 02:45:39
Diff0 days 02:34:42
Diff0 days 11:11:19
Diff0 days 02:48:06
Diff0 days 02:53:53
Diff0 days 02:18:27
Diff0 days 02:25:11
Diff2 days 12:21:52
Diff0 days 02:10:30
Diff0 days 02:46:09
Diff1 days 13:17:51
Diff0 days 03:14:30
Diff0 days 03:47:32
Diff0 days 19:35:27
Diff0 days 02:53:01
Diff0 days 03:28:55
Diff0 days 17:49:06
Diff0 days 02:26:26
Diff0 days 04:25:59
Diff0 days 02:58:25
Diff0 days 02:

In [6]:
if len(stops["stop_id"]) == len(set(stops["stop_id"])):
    print("Y")
else:
    print(len(stops["stop_id"]))
    print(len(set(stops["stop_id"])))

Y


In [10]:
temp_dir = 'output'
pd.DataFrame(agency).to_csv(f"{temp_dir}/agency.txt",index=False)
pd.DataFrame(feed_info).to_csv(f"{temp_dir}/feed_info.txt",index=False)
pd.DataFrame(calendar).to_csv(f"{temp_dir}/calendar.txt",index=False)
pd.DataFrame(stops).to_csv(f"{temp_dir}/stops.txt",index=False)
pd.DataFrame(stop_times).to_csv(f"{temp_dir}/stop_times.txt",index=False)
pd.DataFrame(trips).to_csv(f"{temp_dir}/trips.txt",index=False)
pd.DataFrame(routes).to_csv(f"{temp_dir}/routes.txt",index=False)
pd.DataFrame(shapes).to_csv(f"{temp_dir}/shapes.txt",index=False)

with zipfile.ZipFile("gtfs_output.zip", "w") as zip_file:
    for filename in ["agency.txt","feed_info.txt","stop_times.txt", "stops.txt", "trips.txt", "routes.txt","calendar.txt","shapes.txt"]:
        zip_file.write(f"{temp_dir}/{filename}", arcname=filename)